In [1]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
from pathlib import Path
import pyodbc
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

DATA = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/MIS_REPORT_2022/MONTHLY_REPORTS_2022/DATA')
ASSETS = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/MIS_REPORT_2022/MONTHLY_REPORTS_2022/MR_202211/ASSETS')
IMAGES = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/MIS_REPORT_2022/MONTHLY_REPORTS_2022/MR_202211/IMAGES')

msc=11

# ETL

## Pobieranie danych

In [2]:
# ZAPYTANIE SQL

t_sql="SELECT Konta.Symbol, Konta.Nazwa, ZapisyKsiegowe.NumerDokumentu,ZapisyKsiegowe.Opis,ZapisyKsiegowe.Data, ZapisyKsiegowe.Strona, ZapisyKsiegowe.KwotaZapisuValue FROM ZapisyKsiegowe INNER JOIN Konta ON ZapisyKsiegowe.Konto = Konta.ID AND ZapisyKsiegowe.Okres = Konta.Okres WHERE (ZapisyKsiegowe.Data >= CONVERT(DATETIME, '2022-01-01 00:00:00', 102))"


cnxn=pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                    'Server=192.168.1.252\ENOVA_KOPALNIA;'
                    'Database=OGORZELEC;'
                    'UID=ro_daniel.wojcik;'
                    'PWD=661594050xkr@;'
                    'Trusted_Connection=no;')

t_zapisy=pd.read_sql_query (t_sql,cnxn)
t_zapisy.to_csv(DATA / 'MIS_Zapisy_SQL.csv',index=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Przetwarzanie danych

In [3]:
# Pobiera dane wejściowe MAC


t_plan=pd.read_pickle(DATA /'B22_OUT_PLAN_FINANSOWY.pkl')
t_mapa=t=pd.read_excel(DATA /'Mapa_Kont.xlsx')
t_zapisy=pd.read_csv(DATA /'MIS_Zapisy_SQL.csv')

t_plan_agg=t_plan.groupby(['Konto','M'],as_index=False)['P_2022'].sum()
t_plan_agg['P_2022']=t_plan_agg['P_2022']/1000
t_zapisy['Data']=pd.to_datetime(t_zapisy['Data'], infer_datetime_format=True)

In [4]:
# Pobiera wykonanie z zapisów księgowych


t_zapisy_pl=t_zapisy[(t_zapisy['Symbol'].str.startswith('4')|t_zapisy['Symbol'].str.startswith('7'))].copy()
t_zapisy_pl['Kwota_PLN']=t_zapisy_pl.apply(lambda x:-x['KwotaZapisuValue'] if x['Strona']==1 else x['KwotaZapisuValue'],axis=1)
t_zapisy_pl.drop(['Strona'],axis=1,inplace=True)
t_zapisy_pl.rename(columns={"Symbol":'Konto','NumerDokumentu':'N_Dokument'},inplace=True)
t_zapisy_pl['Data_M']=t_zapisy_pl['Data']+MonthEnd(0)
t_zapisy_pl['R']=t_zapisy_pl['Data'].dt.year
t_zapisy_pl['M']=t_zapisy_pl['Data'].dt.month
t_2022=t_zapisy_pl[t_zapisy_pl['R']==2022]
t_2022=t_2022.groupby(['Konto','M'],as_index=False)[['Kwota_PLN']].sum()
t_2022.rename(columns={'Kwota_PLN':'K_2022'},inplace=True)
t_2022['K_2022']=t_2022['K_2022']/1000

In [5]:
# Buduje mapę kont

t_konta_plan=t_plan_agg[['Konto']].copy()
t_konta_plan.drop_duplicates(subset=['Konto'],keep='last',inplace=True)
t_konta_wykon=t_2022[['Konto']].copy()
t_konta_wykon.drop_duplicates(subset=['Konto'],keep='last',inplace=True)

t_konta_mis=pd.concat([t_konta_plan,t_konta_wykon],ignore_index=True)
t_konta_mis.drop_duplicates(subset=['Konto'],keep='last',inplace=True)
t_konta_mis=pd.merge(t_konta_mis,t_mapa,on=['Konto'],how='left')
t_konta_mis[t_konta_mis['P0'].isnull()==True]

,Konto,Nazwa (FK),Konto_N,P0,P1,P2


In [6]:
#t_zapisy[t_zapisy['Symbol']=='751-003-13']

In [7]:
# Aktualizuje zapisy

t_zapisy=t_zapisy[(t_zapisy['Symbol'].str.startswith('4')|t_zapisy['Symbol'].str.startswith('7'))].copy()
t_zapisy['KwotaZapisuValue']=t_zapisy.apply(lambda x:-x['KwotaZapisuValue'] if x['Strona']==1 else x['KwotaZapisuValue'],axis=1)
t_zapisy.drop(['Strona'],axis=1,inplace=True)
t_zapisy.rename(columns={"Symbol":'Konto','NumerDokumentu':'N_Dokument','KwotaZapisuValue':'Kwota_PLN'},inplace=True)
t_zapisy['Data_M']=t_zapisy['Data']+MonthEnd(0)
t_zapisy['R']=t_zapisy['Data'].dt.year
t_zapisy['M']=t_zapisy['Data'].dt.month

t_zapisy=pd.merge(t_zapisy,t_konta_mis,on='Konto',how='left')

In [8]:
# Buduje raport odchyleń

om=pd.DataFrame({'M':range(1,13)})
t_konta_mis=pd.merge(t_konta_mis,om,how='cross')
t_konta_mis=pd.merge(t_konta_mis,t_plan_agg,on=['Konto','M'],how='left')
t_konta_mis=pd.merge(t_konta_mis,t_2022,on=['Konto','M'],how='left')
t_konta_mis.fillna(0,inplace=True)
t_konta_mis['P0']=t_konta_mis['P0'].str.ljust(36,' ')
t_konta_mis['P1']=t_konta_mis['P1'].str.ljust(36,' ')
t_konta_mis['P2']=t_konta_mis['P2'].str.ljust(36,' ')

## Korekty kontrolingowe

In [9]:
# Korekty kontrolingowe

t_konta_mis['Korekta']=0
t_konta_mis['W_2022']=t_konta_mis['K_2022']+t_konta_mis['Korekta']
t_konta_mis.to_csv(DATA /'MIS_Konta_Analiza_2208.csv')



In [10]:
t_konta_mis.head()

,Konto,Nazwa (FK),Konto_N,P0,P1,P2,M,P_2022,K_2022,Korekta,W_2022
0,402-400-03,USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,402-400-03 | USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,B. Koszty działalności operacyjnej,3.2. Usługi obce - ekspl. złoża,Badania i usługi pomiarowe,1,0.00,0.00,0,0.00
1,402-400-03,USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,402-400-03 | USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,B. Koszty działalności operacyjnej,3.2. Usługi obce - ekspl. złoża,Badania i usługi pomiarowe,2,0.00,0.00,0,0.00
2,402-400-03,USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,402-400-03 | USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,B. Koszty działalności operacyjnej,3.2. Usługi obce - ekspl. złoża,Badania i usługi pomiarowe,3,0.00,0.00,0,0.00
3,402-400-03,USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,402-400-03 | USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,B. Koszty działalności operacyjnej,3.2. Usługi obce - ekspl. złoża,Badania i usługi pomiarowe,4,0.00,0.00,0,0.00
4,402-400-03,USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,402-400-03 | USŁ.POZOSTAŁE POMIAROWO-BADAWCZE,B. Koszty działalności operacyjnej,3.2. Usługi obce - ekspl. złoża,Badania i usługi pomiarowe,5,0.00,0.00,0,0.00


# ANALIZA MIESIĘCZNA

## Wynik finansowy

In [11]:
# Wynik finansowy

t_tabela=t_konta_mis[t_konta_mis['M']==msc]
t_tabela=t_tabela.groupby(['P0'],as_index=False)[['P_2022','K_2022','Korekta','W_2022']].sum()
t_tabela.rename(columns={'P0':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]','K_2022':'K[2022]'},inplace=True)
t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='WYNIK (PBT) RAZEM      '
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]
t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela = t_tabela.replace({np.nan: 0})

t_tabela_fv=t_tabela.style.format({'P[2022]':'{0:,.2f}','K[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}','W[2022]':'{0:,.2f}','Korekta':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

#t_tabela

Pozycja,P[2022],K[2022],Korekta,W[2022],Delta,Delta[proc]
A. Przychody netto ze sprzedaży,"3,881.15","4,829.11",0.00,"4,829.11",947.97,24.42
B. Koszty działalności operacyjnej,"-3,557.88","-4,929.68",0.00,"-4,929.68","-1,371.80",38.56
C. Pozostałe przychody operacyjne,0.00,340.81,0.00,340.81,340.81,inf
D. Pozostałe koszty operacyjne,0.00,-335.86,0.00,-335.86,-335.86,-inf
E. Przychody finansowe,0.00,0.00,0.00,0.00,0.00,0.00
F. Koszty finansowe,-22.06,-20.49,0.00,-20.49,1.57,-7.11
WYNIK (PBT) RAZEM,301.21,-116.10,0.00,-116.10,-417.32,-138.55


In [12]:
# Tabela do raportu - księgowo

tabela=t_tabela[['Pozycja','K[2022]','Korekta']].copy()
tabela=tabela.style.format({'K[2022]':'{0:,.2f}','Korekta':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '0_PBT_Księgowość.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '1_PBT_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '1_PBT_Odchylenia.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [13]:

t_temp=t_konta_mis
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]
#t_wykres['P_Ile']=t_wykres['P_Ile'].cumsum()
#t_wykres['W_Ile']=t_wykres['W_Ile'].cumsum()

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - PBT Razem')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '1_PBT.png')
fig.show()

## Przychody ze sprzedaży

In [14]:
# Przychody ze sprzedaży

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('A. Przychody'))]
t_tabela=t_tabela.groupby(['P1'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P1':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)
t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='PRZYCHODY RAZEM        '
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]
t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})
t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
1. Przychody netto,"3,881.15","4,666.35",785.20,20.23
2. Zmiana stanu produktów,0.00,162.77,162.77,inf
3. Koszt wytworzenia prod. na wł.,0.00,0.00,0.00,0.00
4. Przychody netto TiM,0.00,0.00,0.00,0.00
PRZYCHODY RAZEM,"3,881.15","4,829.11",947.97,24.42


In [15]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '2_Przychody_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '2_Przychody_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')


In [16]:

# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('A. Przychody')==True)]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]
#t_wykres['P_Ile']=t_wykres['P_Ile'].cumsum()
#t_wykres['W_Ile']=t_wykres['W_Ile'].cumsum()

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Przychody')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '2_Przychody.png')
fig.show()

In [17]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('A. Przychody'))&
    (t_zapisy['P1'].str.startswith('1. Przychody netto'))]
t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==8].sort_values(by=['Kwota_PLN'],ascending=False).head(10)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
8184,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/09/0011 Kruszywo 8/11 mm,2022-08-29,8,"432,996.20"
7477,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,"FV FV/22/08/0181 Kruszywo 0/31,5 mm",2022-08-25,8,"245,619.00"
12842,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/09/0028 Kruszywo 11/16 mm,2022-08-31,8,"176,062.27"
12841,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/09/0028 Kruszywo 8/11 mm,2022-08-31,8,"132,128.40"
12838,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/09/0028 Kruszywo 0/2 mm,2022-08-31,8,"128,126.12"
12840,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/09/0028 Kruszywo 4/8 mm,2022-08-31,8,"127,931.73"
12806,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,"FV FV/22/08/0161 Kruszywo 0/31,5 mm",2022-08-22,8,"118,077.20"
11649,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/08/0169 Kruszywo 11/16 mm,2022-08-23,8,"114,573.53"
12843,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,FV FV/22/09/0028 Kruszywo 16/22 mm,2022-08-31,8,"114,443.57"
7478,Sprzedaż krajowa,700-100 | SPRZEDAŻ WYROBÓW,"FV FV/22/08/0181 Kruszywo 0/31,5 mm",2022-08-25,8,"107,426.25"


## Koszty operacyjne

In [18]:
# Koszty operacyjne

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))]
t_tabela=t_tabela.groupby(['P1'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P1':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY OPERACYJNE RAZEM'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
1. Amortyzacja,130.05,129.68,-0.37,-0.29
2. Zużycie materiałów i energii,393.65,730.69,337.05,85.62
3.1. Usługi obce - transport,853.24,"1,681.82",828.57,97.11
3.2. Usługi obce - ekspl. złoża,747.86,"1,013.96",266.11,35.58
3.3. Usługi obce - bocznica,426.46,259.08,-167.38,-39.25
3.4. Usługi obce - pozostałe,375.83,417.06,41.24,10.97
4. Podatki i opłaty,203.01,233.32,30.31,14.93
5. Wynagrodzenia,283.44,313.14,29.70,10.48
6. Ubezpieczenia społeczne,74.92,70.51,-4.41,-5.89
7. Pozostałe koszty rodzajowe,69.42,80.41,10.99,15.83


In [19]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '3_Koszty_Wartość.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '3_Koszty_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [20]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')==True)]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty operacyjne')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '3_Koszty.png')
fig.show()

### Zużycie materiałów i energii

In [21]:
# Koszty operacyjne - zużycie materiałów i energii

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE   '
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Energia elektryczna,161.14,265.85,104.71,64.98
Materiały biurowe,1.49,4.97,3.48,234.00
Materiały pozostałe,0.78,1.04,0.26,32.80
Materiały produkcyjne,146.49,283.81,137.32,93.74
Paliwo,83.75,175.03,91.28,108.99
KOSZTY - ZUŻYCIE MiE,393.65,730.69,337.05,85.62


In [22]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '4_ZMiE_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '4_ZMiE_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [23]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')==True)& (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii')==True)]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty zużycia materiałów i energii')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '4_ZMiE.png')
fig.show()

##### Energia elektryczna

In [24]:
# Wykres energia elektryczna

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')==True)&
    (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii')==True)&
    (t_konta_mis['P2'].str.startswith('Energia elektryczna')==True)]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty energii elektryczne')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '4_ZMiE_EE.png')
fig.show()

In [25]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('2. Zużycie materiałów i energii'))&
    (t_zapisy['P2'].str.startswith('Energia elektryczna'))]
t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
13658,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076112,2022-11-30,11,"-141,690.53"
13660,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076112,2022-11-30,11,"-28,010.53"
13652,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 1000KW D/D1/0344078/22,2022-11-30,11,"-22,053.18"
13659,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076112,2022-11-30,11,"-18,085.56"
13653,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076111,2022-11-03,11,"-15,742.99"
13651,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022\n670KW D/D1/0344215/22,2022-11-30,11,"-10,756.83"
13661,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076112,2022-11-30,11,"-8,892.23"
13662,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076112,2022-11-30,11,"-7,558.39"
13648,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 250KW D/D1/0344214/22,2022-11-30,11,"-3,847.49"
13655,Energia elektryczna,401-200-01 | ZUŻYCIE ENERGII,ENERGIA ELEKTRYCZNA 11/2022 EE/12/22/076111,2022-11-03,11,"-3,112.17"


#### Paliwo

In [26]:
# Wykres 

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')==True)&
    (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii')==True)&
    (t_konta_mis['P2'].str.startswith('Paliwo'))]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty paliwa')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '4_ZMiE_Paliwo.png')
fig.show()

In [27]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('2. Zużycie materiałów i energii'))&
    (t_zapisy['P2'].str.startswith('Paliwo'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
11857,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/070 RW DOLTECH,2022-11-30,11,"-66,836.44"
14368,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/072 RW CAT III,2022-11-30,11,"-51,308.62"
14369,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/073 RW- CAT 980H,2022-11-30,11,"-44,626.26"
14370,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/074 RW Ł200,2022-11-30,11,-830.50
11858,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/071 RW- CASE,2022-11-30,11,-587.05
13482,Paliwo,401-100-05 | ZUŻYCIE PALIWA,"BENZYNA, SUPERB WPR 1610S F 16676K2/4327/22",2022-11-10,11,-354.43
11878,Paliwo,401-100-05 | ZUŻYCIE PALIWA,"DIESEL, OLEJ, PŁYN SKODA OCTAVIA DW3MH48 F 18961K2/0335/22",2022-11-14,11,-349.63
14371,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/075 RW- FORD RANGER,2022-11-30,11,-345.94
11876,Paliwo,401-100-05 | ZUŻYCIE PALIWA,"DIESEL, SKODA OCTAVIA DSW99879- SAMOCHÓD ZASTĘPCZY F 16068K3/0081/22",2022-11-03,11,-343.38
11882,Paliwo,401-100-05 | ZUŻYCIE PALIWA,"BENZYNA, SUPERB WPR 1610S F 7922K1/4407/22",2022-11-09,11,-313.60


In [28]:
# ANALIZA SZCZEGÓŁOWA DOLTECH

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('2. Zużycie materiałów i energii'))&
    (t_zapisy['P2'].str.startswith('Paliwo'))&
    (t_zapisy['Opis'].str.contains('DOLTECH'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza.sort_values(by=['Kwota_PLN'],ascending=True).head(10)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
14090,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/056 RW- DOLTECH,2022-09-30,9,"-68,573.04"
11857,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/070 RW DOLTECH,2022-11-30,11,"-66,836.44"
11602,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/050 RW-DOLTECH,2022-08-31,8,"-61,964.30"
10195,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/063 RW- DOLTECH,2022-10-31,10,"-57,265.23"
6213,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/028 RW- DOLTECH,2022-05-31,5,"-37,508.47"
5941,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/044 RW-DOLTECH,2022-07-31,7,"-36,553.55"
7050,Paliwo,401-100-05 | ZUŻYCIE PALIWA,RW/22/036 RW-DOLTECH,2022-06-30,6,"-33,483.60"


#### Materiały

In [29]:
# Wykres 

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')==True)&
    (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii')==True)&
    (t_konta_mis['P2'].str.contains('Materiały'))]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty materiałów razem')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '4_ZMiE_Materiały.png')
fig.show()

In [30]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('2. Zużycie materiałów i energii'))&
    (t_zapisy['P2'].str.contains('Materiały'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True).head(10)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
14798,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,BLACHY (S)FS-5322/2022/FSW,2022-11-23,11,"-84,554.40"
14293,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,WAŁ MIMOŚRODOWY 020/11/2022,2022-11-14,11,"-67,700.00"
13724,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,SÓL DROGOWA FS 2243/MG/2022,2022-11-09,11,"-23,400.00"
14310,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,"PŁASZCZ STAŁY, PŁASZCZ RUCHOMY, SEGMENT- S3000, CH430 FVS/2022/11/...",2022-11-22,11,"-16,322.14"
14813,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,PRZEKŁADNIA 17/11/2022,2022-11-30,11,"-15,490.00"
13316,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,KRĄŻNIKI FV/25/11/22,2022-11-07,11,"-9,681.00"
14309,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,"PŁASZCZ STAŁY, PŁASZCZ RUCHOMY, SEGMENT- S3000, CH430 FVS/2022/11/...",2022-11-22,11,"-9,631.44"
14295,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,"SITA PLECIONE, HARFOWE, USŁUGA LOGISTYCZNA FS1/22/03237",2022-11-08,11,"-8,464.22"
14790,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,WAŁ NAPĘDOWY 1124/2022,2022-11-04,11,"-6,660.00"
14806,Materiały produkcyjne,401-100-01 | ZUŻYCIE MATERIAŁÓW,OLEJ CAT 980H 78/11/2022,2022-11-21,11,"-5,009.12"


### Usługi obce transport

In [31]:
# Koszty operacyjne - zużycie materiałów i energii

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.1. Usługi obce - transport'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - TRANSPORT'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Transport kolejowy,826.20,"1,464.73",638.53,77.29
Transport samochodowy,27.04,217.09,190.04,702.74
KOSZTY - TRANSPORT,853.24,"1,681.82",828.57,97.11


In [32]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Transport__Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Transport_Odchylenia.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [33]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')==True)& (t_konta_mis['P1'].str.startswith('3.1. Usługi obce - transport')==True)]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty transportu')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Transport.png')
fig.show()

In [34]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.1. Usługi obce - transport'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True).head(100)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
13487,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0011; FV/22/11/0024 CFCK22/069789/22",2022-11-04,11,"-196,364.63"
13697,Transport kolejowy,402-100-20 | USŁ.TRANSPORTOWE POZOSTAŁE PKP DO REFAKT.,"Włodan, FV/22/11/0088 42/11/2022",2022-11-09,11,"-145,418.00"
12128,Transport kolejowy,402-100-10 | USŁ.TRANSPORTOWE PKP - RAIL POLSKA,"PBDIM Świecie, FV/22/11/0129 SK-1/FV/00139/2022/11",2022-11-16,11,"-110,786.00"
13322,Transport kolejowy,402-100-20 | USŁ.TRANSPORTOWE POZOSTAŁE PKP DO REFAKT.,"Włodan, FV/22/11/0081 39/11/2022",2022-11-08,11,"-107,583.00"
13490,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0126 CFCK22/071966/22",2022-11-14,11,"-105,618.24"
13499,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0207 CFCK22/074659/22",2022-11-28,11,"-105,618.24"
13495,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0163 CFCK22/073107/22",2022-11-21,11,"-105,561.82"
13498,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0171 CFCK22/073684/22",2022-11-23,11,"-105,279.72"
11469,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0071 CFCK22/070194/22",2022-11-07,11,"-105,166.88"
11470,Transport kolejowy,402-100-06 | USŁ.TRANSPORTOWE PKP - CARGO,"Kobylarnia S.A., FV/22/11/0090 CFCK22/070781/22",2022-11-10,11,"-104,269.91"


### Usługi obce obsługa złoża

In [35]:
# Koszty operacyjne 

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - OBSŁUGA ZŁOŻA'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Badania i usługi pomiarowe,9.65,6.83,-2.83,-29.27
Dzierżawa gruntów,97.83,92.72,-5.10,-5.22
Naprawy i remonty,33.45,30.46,-2.98,-8.92
Transport i zał. wewnętrzny,380.31,529.74,149.43,39.29
Usługi wiertniczo - strzałowe,162.44,281.27,118.83,73.16
Wynajem maszyn i urządzeń,64.18,72.94,8.75,13.64
KOSZTY - OBSŁUGA ZŁOŻA,747.86,"1,013.96",266.11,35.58


In [36]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Obsługa__Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Obsługa_Odchylenia.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [37]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty obsługi złoża')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Obsługa.png')
fig.show()

### Naprawy i remonty

In [38]:
# Wykres

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_konta_mis['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_konta_mis['P2'].str.startswith('Naprawy i remonty'))]
    
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - naprawy i remonty')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Naprawy_Remonty.png')
fig.show()

In [39]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_zapisy['P2'].str.startswith('Naprawy i remonty'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
13382,Naprawy i remonty,402-800-03 | USŁ. REMONTOWO-NAPRAWCZE - BOCZNICA,Wymiana iglicy i spony na rozjeździe nr 35 27/11/22/FVS,2022-11-15,11,"-9,000.00"
12200,Naprawy i remonty,402-800-01 | USŁ. REMONTOWO-NAPRAWCZE - MASZYNY,LEGALIZACJA WAG NAJAZDOWYCH FS/2022/11/0314,2022-11-24,11,"-7,500.00"
12201,Naprawy i remonty,402-800-01 | USŁ. REMONTOWO-NAPRAWCZE - MASZYNY,LEGALIZACJA WAG NAJAZDOWYCH FS/2022/11/0314,2022-11-24,11,"-7,500.00"
12197,Naprawy i remonty,402-800-01 | USŁ. REMONTOWO-NAPRAWCZE - MASZYNY,"DOSTAWA, MONTAŻ I WULKANIZACJA TAŚMY VAT/1201/2022",2022-11-23,11,"-3,250.00"
12196,Naprawy i remonty,402-800-01 | USŁ. REMONTOWO-NAPRAWCZE - MASZYNY,"PRZEGLĄD SPRĘŻARKI, MATERIAŁY FAS/2/11/2022",2022-11-04,11,"-1,443.00"
11895,Naprawy i remonty,402-800-01 | USŁ. REMONTOWO-NAPRAWCZE - MASZYNY,KOMPLEKSOWA OBSŁUGA SERWISOWA- WAGI FS/2022/11/0008,2022-11-02,11,-500.00
11896,Naprawy i remonty,402-800-01 | USŁ. REMONTOWO-NAPRAWCZE - MASZYNY,KOMPLEKSOWA OBSŁUGA SERWISOWA- WAGI FS/2022/11/0008,2022-11-02,11,-500.00
15055,Naprawy i remonty,"402-800-04 | USŁ. REMONTOWO-NAPRAWCZE - WAGA, LABO.,ADMIN, WARSZTAT",Sprawdzenie licznika i plombowanie urządzenia pomiarowo-rozliczeni...,2022-11-21,11,-240.10
13701,Naprawy i remonty,402-800-02 | USŁ. REMONTOWO-NAPRAWCZE - SAMOCHODY,WYMIANA OPON DSW99879 360/11/2022,2022-11-17,11,-115.58
13702,Naprawy i remonty,402-800-02 | USŁ. REMONTOWO-NAPRAWCZE - SAMOCHODY,WYMIANA OPON DW3MH48 360/11/2022,2022-11-17,11,-115.58


##### Transport wewnętrzny

In [40]:
# Wykres transport wewnętrzny

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_konta_mis['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_konta_mis['P2'].str.startswith('Transport i zał. wewnętrzny'))]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - transport wewnętrzny')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Obsługa_Transport_IN.png')
fig.show()

In [41]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_zapisy['P2'].str.startswith('Transport i zał. wewnętrzny'))]
t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
11909,Transport i zał. wewnętrzny,402-200-08 | ZAŁAD.I TRANS.UROBKU,ZAŁAD.I TRANS.UROBKU 118/11/2022,2022-11-20,11,"-105,416.00"
13410,Transport i zał. wewnętrzny,402-200-08 | ZAŁAD.I TRANS.UROBKU,ZAŁAD.I TRANS.UROBKU 166/11/2022,2022-11-26,11,"-96,534.90"
13424,Transport i zał. wewnętrzny,402-200-09 | TRANS. KAMIENIA NA PLAC SKŁADOWY,ZAŁAD. I TRANS. KRUSZYWA NA PLAC SKŁAD FA/144/11/2022,2022-11-30,11,"-90,720.00"
11901,Transport i zał. wewnętrzny,402-200-08 | ZAŁAD.I TRANS.UROBKU,ZAŁAD.I TRANS.UROBKU 78/11/2022,2022-11-10,11,"-84,193.35"
13425,Transport i zał. wewnętrzny,402-200-08 | ZAŁAD.I TRANS.UROBKU,ZAŁAD.I TRANS.UROBKU 14/12/2022,2022-11-30,11,"-46,315.26"
11893,Transport i zał. wewnętrzny,402-200-08 | ZAŁAD.I TRANS.UROBKU,ZAŁAD.I TRANS.UROBKU 18/11/2022,2022-11-04,11,"-45,237.76"
13717,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 118/11/2022,2022-11-20,11,"-10,944.00"
14786,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANSPORT WTÓRNY KRUSZYWA FA/144/11/2022,2022-11-30,11,"-9,800.00"
11619,Transport i zał. wewnętrzny,402-200-07 | TRANS. KAMIENIA I ZIEMI,ZAŁAD.I TRANS. KAMIENIA I ZIEMI 118/11/2022,2022-11-20,11,"-8,850.00"
12273,Transport i zał. wewnętrzny,402-200-07 | TRANS. KAMIENIA I ZIEMI,ZAŁAD.I TRANS.KAMIENIA I ZIEMI 166/11/2022,2022-11-26,11,"-7,500.00"


In [42]:
# ANALIZA SZCZEGÓŁOWA WTÓRNY

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_zapisy['P2'].str.startswith('Transport i zał. wewnętrzny'))&
    (t_zapisy['Opis'].str.contains('WTÓRNY'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza.sort_values(by=['Kwota_PLN'],ascending=True).head(10)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
12847,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA FA/76/07/2022,2022-07-29,7,"-46,620.00"
8846,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS. WTÓRNY KRUSZYWA FA/96/06/2022,2022-06-30,6,"-40,180.00"
11247,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 82/07/2022,2022-07-11,7,"-22,392.00"
7886,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 86/04/2022,2022-04-09,4,"-18,600.00"
13308,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANSPORT WTÓRNY KRUSZYWA 73/08/2022,2022-08-07,8,"-18,216.00"
4373,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 216/03/2022,2022-03-25,3,"-18,216.00"
13694,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 93/05/2022,2022-05-13,5,"-17,904.00"
9953,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANSPORT WTÓRNY KRUSZYWA 135/08/2022,2022-08-21,8,"-17,640.00"
8395,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 126/07/2022,2022-07-18,7,"-17,208.00"
4813,Transport i zał. wewnętrzny,402-200-06 | TRANS.WTÓRNY KRUSZ.,TRANS.WTÓRNY KRUSZYWA 10/05/2022,2022-04-30,4,"-17,160.00"


In [43]:
t_analiza.groupby(['M'],as_index=False)[['Kwota_PLN']].sum()

,M,Kwota_PLN
0,1,"-12,864.00"
1,2,"-9,264.00"
2,3,"-56,664.00"
3,4,"-71,370.00"
4,5,"-47,760.00"
5,6,"-97,300.00"
6,7,"-115,244.00"
7,8,"-63,912.00"
8,9,"-51,984.00"
9,10,"-48,576.00"


##### Usługi wiertniczo-strzałowe

In [44]:
# Wykres

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_konta_mis['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_konta_mis['P2'].str.startswith('Usługi wiertniczo - strzałowe'))]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Usługi wiertniczo-strzałowe')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_SSE.png')
fig.show()

In [45]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))&
    (t_zapisy['P2'].str.startswith('Usługi wiertniczo - strzałowe'))]
t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==msc].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
12275,Usługi wiertniczo - strzałowe,402-500-01 | USŁ. WIERTNICZO-STRZAŁOWE,ROBOTY WIERTNICZO STRZAŁOWE PL4229000872,2022-11-30,11,"-160,922.16"
11617,Usługi wiertniczo - strzałowe,402-500-01 | USŁ. WIERTNICZO-STRZAŁOWE,USŁUGI WIERTNICZO-STRZAŁOWE PL4229000825,2022-11-15,11,"-120,350.55"


### Usługi obce bocznica

In [46]:
# Koszty operacyjne

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.3. Usługi obce - bocznica'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - OBSŁUGA BOCZNICY'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Transport i obsługa bocznicy,408.68,239.18,-169.50,-41.47
Wynajem bocznicy,17.78,19.90,2.12,11.91
KOSZTY - OBSŁUGA BOCZNICY,426.46,259.08,-167.38,-39.25


In [47]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Bocznica__Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Bocznica_Odchylenia.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [48]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.3. Usługi obce - bocznica'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty obsługi i transportu na bocznicę')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Bocznica.png')
fig.show()

### Usługi obce pozostałe

In [49]:
# Koszty operacyjne

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.4. Usługi obce - pozostałe'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - USŁUGI OBCE POZOSTAŁE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Leasing operacyjny,10.53,0.03,-10.49,-99.69
Monitoring TMS,1.61,2.00,0.39,24.52
Ochrona,21.43,-0.00,-21.43,-100.00
"Poczta, telefon, internet",1.60,1.84,0.25,15.50
Pozostałe usługi,335.20,406.60,71.41,21.30
"Telefon, internet",1.15,0.95,-0.21,-17.79
Usługi IT,4.32,5.64,1.32,30.44
KOSZTY - USŁUGI OBCE POZOSTAŁE,375.83,417.06,41.24,10.97


In [50]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Pozostałe__Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '5_UO_Pozostałe_Odchylenia.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [51]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.4. Usługi obce - pozostałe'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Usługi obce pozostałe')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Pozostałe.png')
fig.show()

In [52]:
t_wykres.describe()

,M,P_2022,W_2022
count,11.00,11.00,11.00
mean,6.00,345.08,340.47
std,3.32,45.34,72.96
min,1.00,243.40,224.55
25%,3.50,335.98,285.82
50%,6.00,369.45,363.47
75%,8.50,375.56,394.66
max,11.00,381.88,440.00


#### Pozostałe usługi

In [53]:
# Wykres 

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_konta_mis['P1'].str.startswith('3.4. Usługi obce - pozostałe'))&
    (t_konta_mis['P2'].str.startswith('Pozostałe usługi'))]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Pozostałe usługi')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '5_UO_Pozostałe_Pozostałe.png')
fig.show()

In [54]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.4. Usługi obce - pozostałe'))&
    (t_zapisy['P2'].str.startswith('Pozostałe usługi'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']<=msc].sort_values(by=['Kwota_PLN'],ascending=True).head(20)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
14186,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI Z GÓRNYCH PARTII CELEM UDOST. PARTII ZŁOŻOWYCH...,2022-08-31,8,"-194,400.00"
13829,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 222/01/09/2022,2022-09-30,9,"-187,110.00"
13420,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 267/01/11/2022,2022-11-30,11,"-163,400.00"
11416,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 246/01/10/2022,2022-10-28,10,"-147,830.00"
4285,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,KRUSZENIE WSTĘPNE KRUSZARKĄ SZCZĘKOWĄ 154/01/05/2022,2022-05-31,5,"-135,000.00"
8713,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 190/01/07/2022,2022-07-29,7,"-116,640.00"
71,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,WYKONANIE PROJEKTU WAŁU - WK3 FS/14/SO/01/2022,2022-01-12,1,"-104,000.00"
4496,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 109/01/03/2022,2022-03-31,3,"-102,870.00"
6525,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,KRUSZENIE WSTĘPNE KRUSZARKĄ SZCZĘKOWĄ-AWARIA KRUSZARKI 44.4 166/01...,2022-06-30,6,"-82,800.00"
4269,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 43/05/22/FVS,2022-05-21,5,"-73,425.05"


In [55]:
# ANALIZA SZCZEGÓŁOWA KRUSZENIE AWARIA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.4. Usługi obce - pozostałe'))&
    (t_zapisy['P2'].str.startswith('Pozostałe usługi'))&
    (t_zapisy['Opis'].str.contains('KRUSZENIE WSTĘPNE'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza.sort_values(by=['Kwota_PLN'],ascending=True)
t_analiza.loc['R']=t_analiza.sum(numeric_only=True,axis=0)
t_analiza.loc['R','P2']='PODSUMOWANIE'


t_analiza

,P2,Konto_N,Opis,Data,M,Kwota_PLN
4285,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,KRUSZENIE WSTĘPNE KRUSZARKĄ SZCZĘKOWĄ 154/01/05/2022,2022-05-31,5.00,"-135,000.00"
6525,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,KRUSZENIE WSTĘPNE KRUSZARKĄ SZCZĘKOWĄ-AWARIA KRUSZARKI 44.4 166/01...,2022-06-30,6.00,"-82,800.00"
R,PODSUMOWANIE,NaN,NaN,NaT,11.00,"-217,800.00"


In [56]:
# ANALIZA SZCZEGÓŁOWA PRODUKCJA MIESZANKI

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.4. Usługi obce - pozostałe'))&
    (t_zapisy['P2'].str.startswith('Pozostałe usługi'))&
    (t_zapisy['Opis'].str.contains('MIESZANK'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza = t_analiza.sort_values(by=['Kwota_PLN'],ascending=True)
#t_analiza.loc['R']=t_analiza.sum(numeric_only=True,axis=0)
#t_analiza.loc['R','P2']='PODSUMOWANIE'


t_analiza.head()

,P2,Konto_N,Opis,Data,M,Kwota_PLN
14186,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI Z GÓRNYCH PARTII CELEM UDOST. PARTII ZŁOŻOWYCH...,2022-08-31,8,"-194,400.00"
13829,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 222/01/09/2022,2022-09-30,9,"-187,110.00"
13420,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 267/01/11/2022,2022-11-30,11,"-163,400.00"
11416,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 246/01/10/2022,2022-10-28,10,"-147,830.00"
8713,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,PRODUKCJA MIESZANKI 190/01/07/2022,2022-07-29,7,"-116,640.00"


In [57]:
temp_stat=t_analiza.groupby(['M'],as_index=False)[['Kwota_PLN']].sum()
temp_stat

,M,Kwota_PLN
0,1,"-123,063.76"
1,2,"-204,391.26"
2,3,"-308,533.52"
3,4,"-294,901.61"
4,5,"-214,528.85"
5,6,"-61,560.00"
6,7,"-116,640.00"
7,8,"-194,400.00"
8,9,"-187,110.00"
9,10,"-147,830.00"


In [58]:
temp_stat.describe()

,M,Kwota_PLN
count,11.00,11.00
mean,6.00,"-183,305.36"
std,3.32,"73,649.77"
min,1.00,"-308,533.52"
25%,3.50,"-209,460.05"
50%,6.00,"-187,110.00"
75%,8.50,"-135,446.88"
max,11.00,"-61,560.00"


In [59]:
# ANALIZA SZCZEGÓŁOWA INNE NIZ PRODUKCJA MIESZANKI

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('3.4. Usługi obce - pozostałe'))&
    (t_zapisy['P2'].str.startswith('Pozostałe usługi'))&
    ((t_zapisy['Opis'].str.contains('MIESZANK')==False) & (t_zapisy['Opis'].str.contains('WSTĘPNE')==False))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza=t_analiza.sort_values(by=['Kwota_PLN'],ascending=True)
#t_analiza.loc['R']=t_analiza.sum(numeric_only=True,axis=0)
#t_analiza.loc['R','P2']='PODSUMOWANIE'


t_analiza.head(10)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
71,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,WYKONANIE PROJEKTU WAŁU - WK3 FS/14/SO/01/2022,2022-01-12,1,"-104,000.00"
8758,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,WYK.OKŁADZIN PODAWACZA 448/2022,2022-09-20,9,"-30,600.00"
1956,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,"OTWÓR NR 1- POSZUKIWAWCZY, GŁĘBOKOŚĆ 40M FV/2/01/2022",2022-01-24,1,"-28,000.00"
1961,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,WYKONANIE WSYPU NA KRUSZARKĘ- WYMIANA 33/2022,2022-01-28,1,"-26,800.00"
13412,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,OCHRONA MIENIA M-C 11/22 FV/63/11/22/ochrona,2022-11-30,11,"-21,696.00"
12212,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,USŁ. DORADCZE- ZARZĄD 08/2022 FVS/1/08/2022,2022-08-31,8,"-21,500.00"
8776,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,USŁ. DORADCZE- ZARZĄD FVS/1/07/2022,2022-09-29,9,"-21,500.00"
12205,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,USŁ. DORADCZE- ZARZĄD A1/11/2022,2022-11-21,11,"-21,500.00"
9197,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,USŁ. DORADCZE- ZARZĄD FVS/1/10/2022,2022-10-20,10,"-21,500.00"
8768,Pozostałe usługi,402-600-02 | USŁ. OBCE-POZOSTAŁE,USŁ. DORADCZE- ZARZĄD FVS/1/09/2022,2022-09-23,9,"-21,500.00"


### Podatki i opłaty

In [60]:
# Koszty operacyjne - Podatki i opłaty

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('4. Podatki i opłaty'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - PODATKI I OPŁATY'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Opłata eksploatacyjna,112.43,150.12,37.69,33.52
Opłaty środowiskowe,0.70,0.05,-0.65,-93.16
PFRON,3.39,4.87,1.48,43.58
Podatek leśny,-0.00,-0.00,0.00,0.00
Podatek od nieruchomości,32.20,32.80,0.60,1.86
Podatek rolny,-0.00,-0.00,0.00,0.00
Pozostałe opłaty,0.50,0.53,0.03,6.51
Uzytkowanie gruntów leśnych,53.78,44.94,-8.83,-16.43
Użytkowanie wieczyste,-0.00,-0.00,0.00,0.00
KOSZTY - PODATKI I OPŁATY,203.01,233.32,30.31,14.93


In [61]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '6_Podatki_Opłaty_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '6_Podatki_Opłaty_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [62]:
# Wykres

t_temp=t_konta_mis[t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej ')& (t_konta_mis['P1'].str.startswith('4. Podatki i opłaty'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Podatki i opłaty')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '6_Podatki_Opłaty.png')
fig.show()

#### Opłata eksploatacyjna

In [63]:
# Wykres 

t_temp=t_konta_mis[
    (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_konta_mis['P1'].str.startswith('4. Podatki i opłaty'))&
    (t_konta_mis['P2'].str.startswith('Opłata eksploatacyjna'))]

t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty opłat eksploatacyjnych')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '6_Podatki_Opłaty_OE.png')
fig.show()

In [64]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('4. Podatki i opłaty'))&
    (t_zapisy['P2'].str.startswith('Opłata eksploatacyjna'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza.sort_values(by=['M'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
214,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 01 2022,2022-01-31,1,"-40,438.92"
586,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 02 2022,2022-02-28,2,"-85,259.97"
3756,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 03 2022,2022-03-31,3,"-161,815.02"
7551,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 04 2022,2022-04-30,4,"-126,522.71"
6556,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 05 2022,2022-05-31,5,"-107,688.30"
7933,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 06 2022,2022-06-30,6,"-137,388.68"
10618,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA 07 2022,2022-07-31,7,"-184,755.40"
10620,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,PRZEKS Z NKUP NA KUP- OPŁATA EKSPL. I PÓŁROCZE 2022,2022-07-31,7,"658,114.00"
11245,Opłata eksploatacyjna,403-011-01 | OPŁ. EKSPLOATACYJNA KUP,PRZEKS Z NKUP NA KUP- OPŁATA EKSPL. I PÓŁROCZE 2022,2022-07-31,7,"-658,114.00"
13873,Opłata eksploatacyjna,403-011-02 | OPŁ. EKSPLOATACYJNA NKUP,OPŁATA EKSPLOATACYJNA ZA M-C SIERPIEŃ 2022R.,2022-08-31,8,"-149,006.89"


## Wynagrodzenia

In [65]:
# Koszty operacyjne - Wynagrodzenia

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('5. Wynagrodzenia'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - WYNAGRODZENIA'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')
t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Ekwiwalenty i odprawy,-0.00,12.84,12.84,-inf
Nagrody jubileuszowe,-0.00,-0.00,0.00,0.00
Rada nadzorcza,7.50,7.50,0.00,0.00
Rezerwa nagrody jubileuszowe,-0.00,-0.00,0.00,0.00
Rezerwa nagrody poozstałe,-0.00,-0.00,0.00,0.00
Rezerwa urlopowa,-0.00,-0.00,0.00,0.00
Umowy o pracę,261.99,279.24,17.25,6.59
Umowy zlecenia,12.75,7.75,-5.00,-39.18
Zarząd i Prokura,1.20,5.80,4.60,383.33
KOSZTY - WYNAGRODZENIA,283.44,313.14,29.70,10.48


In [66]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '7_Wynagrodzenia_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '7_Wynagrodzenia__Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [67]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('5. Wynagrodzenia'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty wynagrodzeń')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '7_Wynagrodzenia.png')
fig.show()

In [68]:
t_wykres.describe()

,M,P_2022,W_2022
count,11.00,11.00,11.00
mean,6.00,289.73,303.27
std,3.32,26.41,53.79
min,1.00,230.43,232.60
25%,3.50,283.14,260.76
50%,6.00,289.33,311.83
75%,8.50,293.59,342.08
max,11.00,334.01,394.09


In [69]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('5. Wynagrodzenia'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza.sort_values(by=['M'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
24,Umowy zlecenia,404-006-02 | WYNAGRODZENIA-BEZOSOB.FUND.PŁAC NKUP,LPU/F/2022/01/4,2022-01-31,1,"-10,135.00"
1170,Rada nadzorcza,404-005-01 | WYNAGRODZENIA-RADA NADZORCZA KUP,LPU/F/2022/01/1,2022-01-31,1,"-7,500.00"
223,Umowy o pracę,404-001 | WYNAGRODZENIA,LPE/ADM/2022/01/1,2022-01-31,1,"-72,200.50"
218,Umowy o pracę,404-001 | WYNAGRODZENIA,LPE/WAR/2022/01/1,2022-01-31,1,"-11,545.21"
217,Umowy o pracę,404-001 | WYNAGRODZENIA,LPE/ZAK/2022/01/1,2022-01-31,1,"-19,037.33"
219,Umowy o pracę,404-001 | WYNAGRODZENIA,LPE/DOZ/2022/01/1,2022-01-31,1,"-31,989.00"
2903,Ekwiwalenty i odprawy,404-003-01 | WYNAGRODZENIA-EKWIWALENT ZA URLOP,LPE/ZDP/2022/01/1,2022-01-31,1,-393.44
97,Zarząd i Prokura,404-007 | WYNAGRODZENIA-DZIAŁ.WYK.OSOBIŚCIE,LPU/F/2022/01/2,2022-01-31,1,"-7,500.00"
95,Zarząd i Prokura,404-007 | WYNAGRODZENIA-DZIAŁ.WYK.OSOBIŚCIE,LPU/F/2022/01/3,2022-01-31,1,"-1,200.00"
74,Rezerwa urlopowa,404-003-02 | WYNAGRODZENIA-EKWIWALENT ZA URLOP NKUP (REZERWA),LPE/ZDP/2022/01/1,2022-01-31,1,"-1,770.48"


### ZUS

In [70]:
# Koszty operacyjne - ZUS

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('6. Ubezpieczenia społeczne'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUS I ŚWIADCZENIA'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Badania i opieka medyczna,1.51,0.42,-1.09,-71.92
Badania lekarskie,0.69,1.20,0.51,73.52
Ekwiwalenty,0.99,0.12,-0.87,-87.56
Fundusz socjalny,6.23,6.69,0.46,7.44
PPK,0.20,0.41,0.21,102.19
Rezerwa ZUS,-0.00,-0.00,0.00,0.00
Składki ZUS,55.21,55.51,0.29,0.53
Szkolenia,4.13,3.61,-0.52,-12.64
Świadczenia na rzecz pracowników,5.95,2.55,-3.41,-57.19
KOSZTY - ZUS I ŚWIADCZENIA,74.92,70.51,-4.41,-5.89


In [71]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '8_ZUS_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '8_ZUS_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [72]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('6. Ubezpieczenia społeczne'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty narzutów na wynagrodzenia')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '8_ZUS.png')
fig.show()

In [73]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('6. Ubezpieczenia społeczne'))&
    (t_zapisy['Opis'].str.contains('ZUS'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==8].sort_values(by=['Kwota_PLN'],ascending=True).head()

,P2,Konto_N,Opis,Data,M,Kwota_PLN
13910,Składki ZUS,405-001-02 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP,ZUS firma LPE/ZDP/2022/08/1,2022-08-31,8,"-11,333.10"
13911,Składki ZUS,405-001-02 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP,ZUS firma LPE/DOZ/2022/08/1,2022-08-31,8,"-8,890.25"
13905,Składki ZUS,405-001-02 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP,ZUS firma GN/ZDP/2022/08/1,2022-08-31,8,"-8,535.02"
13909,Składki ZUS,405-001-02 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP,ZUS firma LPE/ADM/2022/08/1,2022-08-31,8,"-7,013.32"
13902,Składki ZUS,405-001-02 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP,ZUS firma LPE/WAR/2022/08/1,2022-08-31,8,"-4,618.16"


In [74]:
t_analiza.groupby(['M'],as_index=False)[['Kwota_PLN']].sum()

,M,Kwota_PLN
0,1,"-46,643.12"
1,2,"-62,870.62"
2,3,"-38,911.76"
3,4,"-46,714.70"
4,5,"-21,442.87"
5,6,"-54,218.01"
6,7,"-43,222.65"
7,8,"-66,868.13"
8,9,"-61,155.75"
9,10,"-72,292.76"


In [75]:
# ANALIZA NIE ZUS

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('6. Ubezpieczenia społeczne'))&
    (t_zapisy['Opis'].str.contains('ZUS')==False)]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza.sort_values(by=['Kwota_PLN'],ascending=True).head(10)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
6557,Rezerwa ZUS,405-001-04 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP (REZERWA),LPE/ADM/2022/05/1,2022-05-31,5,"-19,006.76"
2714,Rezerwa ZUS,405-001-04 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP (REZERWA),ROZWIĄZANIE REZERWY 2021 Z TYT.NAGRODY,2022-02-28,2,"-18,279.41"
4073,Świadczenia na rzecz pracowników,405-002 | ŚWIADCZENIA DLA PRACOWNIKÓW,"UBRANIA ROBOCZE:KOSZULA, BLUZA, SPODNIE, BLUZA, UBRANIE SZWEDZKIE ...",2022-02-28,2,"-12,351.00"
12316,Świadczenia na rzecz pracowników,405-002 | ŚWIADCZENIA DLA PRACOWNIKÓW,UBRANIE ROBOCZE BHP-ZIMOWE 367/O/01/2022,2022-09-20,9,"-11,713.00"
8656,Składki ZUS,405-001-02 | SKŁADKI Z TYT.UBEZPIECZEŃ - ZUS NKUP,GN/ZDP/2022/07/1,2022-07-31,7,"-10,008.06"
1370,Fundusz socjalny,405-005-02 | NALICZONY FUNDUSZ SOCJALNY NKUP,FUNDUSZ SOCJALNY,2022-03-31,3,"-6,694.49"
8500,Fundusz socjalny,405-005-02 | NALICZONY FUNDUSZ SOCJALNY NKUP,FUNDUSZ SOCJALNY,2022-07-31,7,"-6,694.49"
1168,Fundusz socjalny,405-005-02 | NALICZONY FUNDUSZ SOCJALNY NKUP,FUNDUSZ SOCJALNY,2022-01-31,1,"-6,694.49"
14464,Fundusz socjalny,405-005-02 | NALICZONY FUNDUSZ SOCJALNY NKUP,FUNDUSZ SOCJALNY,2022-11-30,11,"-6,694.49"
14112,Fundusz socjalny,405-005-02 | NALICZONY FUNDUSZ SOCJALNY NKUP,FUNDUSZ SOCJALNY,2022-09-30,9,"-6,694.49"


### Pozostałe koszty operacyjne

In [76]:
# Koszty operacyjne - pozostałe

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('7. Pozostałe koszty rodzajowe'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']


t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - POZOSTAŁE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Fundusz Likwidacji Z. Górniczego,11.24,15.00,3.76,33.45
Podróże służbowe,0.87,2.24,1.38,158.91
Pozostałe koszty,0.11,0.91,0.80,727.66
Repreezntacja i reklama,6.50,23.05,16.55,254.68
Rozliczenie projektów rozwojowych,28.68,14.37,-14.32,-49.91
Ryczałty samochodowe,0.22,0.46,0.24,106.90
Samochody - nieodliczony VAT,4.11,3.81,-0.30,-7.20
Ubezpieczenia komunikacyjne,-0.00,2.15,2.15,-inf
Ubezpieczenia majątkowe,16.61,14.35,-2.26,-13.61
Ubezpieczenia wyrrobów gotowych,-0.00,3.46,3.46,-inf


In [77]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '9_Pozostałe_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '9_Pozostałe_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [78]:
# Wykres

t_temp=t_konta_mis[(t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('7. Pozostałe koszty rodzajowe'))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Koszty operacyjne pozostałe')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '9_Koszty_pozostałe.png')
fig.show()

In [79]:
# ANALIZA

t_analiza=t_zapisy[
    (t_zapisy['P0'].str.startswith('B. Koszty działalności operacyjnej '))&
    (t_zapisy['P1'].str.startswith('7. Pozostałe koszty rodzajowe'))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==8].sort_values(by=['Kwota_PLN'],ascending=True).head()

,P2,Konto_N,Opis,Data,M,Kwota_PLN
13675,Fundusz Likwidacji Z. Górniczego,409-008-02 | POZ. KOSZ.-FUND. LIKW.Z-DU GÓR NKUP,NALICZENIE FLZG 08/2022,2022-08-31,8,"-14,900.69"
14021,Rozliczenie projektów rozwojowych,409-010 | BUD.Z-DU WYDOBYWCZ.NOWE ZŁOŻE NKUP,RMK ROZWÓJ ZŁOŻA 01-08.22 DOT.PKE2022/08/0016,2022-08-31,8,"-3,374.28"
11592,Rozliczenie projektów rozwojowych,409-009 | BUD.Z-DU WYDOBYWCZ.NOWE ZŁOŻE KUP,"PRZYGOT.NOWEJ PROD.ETAP VI, LATA 2019-2050",2022-08-31,8,"-3,115.20"
11351,Ubezpieczenia wyrrobów gotowych,409-003-03 | UBEZPIECZENIA WYROBÓW GOTOWYCH,UBEZPIECZENIA MAJĄTKOWE- UBEZP.OC Z TYT.PROWADZ.DZIAŁ.GOSP. OKRES ...,2022-08-31,8,"-3,004.91"
11581,Rozliczenie projektów rozwojowych,409-009 | BUD.Z-DU WYDOBYWCZ.NOWE ZŁOŻE KUP,"BUDOWA Z-DU WYD.NOWE ZŁOŻE, LATA 2019-2050",2022-08-31,8,"-2,379.26"


## Wynik na pozostałej działalności operacyjnej

In [80]:
# Pozostała działalność

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & ((t_konta_mis['P0'].str.startswith('C. Pozostałe przychody operacyjne'))| (t_konta_mis['P0'].str.startswith('D. Pozostałe koszty operacyjne')))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='WYNIK NA PDO'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Odpisy aktualizujące - zapasy,-0.00,-0.21,-0.21,inf
Odszkodowania i szkody,-0.00,-0.00,0.00,0.00
Pozostałe,-0.00,9.99,9.99,-inf
Rekultywacja,-0.00,0.96,0.96,-inf
Rozwiązanie rezerw,-0.00,-0.00,0.00,0.00
Sprzedaż RAT,-0.00,-15.69,-15.69,inf
WYNIK NA PDO,0.00,-4.95,-4.95,-inf


In [81]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '10_OOP_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '10_OOP_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [82]:
# Wykres

t_temp=t_konta_mis[((t_konta_mis['P0'].str.startswith('C. Pozostałe przychody operacyjne'))| (t_konta_mis['P0'].str.startswith('D. Pozostałe koszty operacyjne')))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

#t_wykres['P_2022']=-t_wykres['P_2022']
#t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Wynik na pozostałej działalności')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '10_OOP.png')
fig.show()

In [83]:
# ANALIZA

t_analiza=t_zapisy[((t_zapisy['P0'].str.startswith('C. Pozostałe przychody operacyjne'))| (t_zapisy['P0'].str.startswith('D. Pozostałe koszty operacyjne')))]

t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==8].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
13689,Odpisy aktualizujące - zapasy,761-013 | P.K.O.ODP.OBNIŻAJĄCE ZAPASY,RWW/22/010 RWW,2022-08-01,8,"-463,730.09"
6015,Pozostałe,761-010-02 | P.K.O.DAROWIZNY,DOFINANSOWANIE DO REMONTU MOSTU - ZGODNIE Z ZAWARTĄ UMOWĄ Z DSDIK/...,2022-08-04,8,"-40,085.66"
13289,Pozostałe,761-012 | P.K.O. POZOSTAŁE nkup,DOK.GEOL.CHEŁMCZYK,2022-08-31,8,"-10,000.00"
14122,Pozostałe,761-001 | P.K.O.ZAL.KOMORNIK OD NALEŻNOŚCI,BOBECKI WITOLD KM 542/15 KSG: 23839/22 SYGN: V GNC UPR 206/14 DATA...,2022-08-31,8,-784.79
15094,Pozostałe,761-012 | P.K.O. POZOSTAŁE nkup,PRZEKS.VAT ZA 08/2022,2022-08-31,8,-0.20
8188,Pozostałe,760-001 | POZOSTAŁE PRZYCH.OPERACYJNE,"OPŁATA ZA FAKTURĘ250947/POS,236362/POS PŁATNE PRZEZ FUNDACJĘ",2022-08-31,8,81.47
6917,Pozostałe,760-003 | P.P.O.ZWROT KOSZTÓW SĄD./KOMORNIK,BOBECKI WITOLD KM 542/15 KSG: 24105/22 SYGN: V GNC UPR 206/14 DATA...,2022-08-22,8,115.32
11400,Pozostałe,761-001 | P.K.O.ZAL.KOMORNIK OD NALEŻNOŚCI,BOBECKI WITOLD KM 542/15 KSG: 23839/22 SYGN: V GNC UPR 206/14 DATA...,2022-08-19,8,784.79
14123,Pozostałe,760-003 | P.P.O.ZWROT KOSZTÓW SĄD./KOMORNIK,BOBECKI WITOLD KM 542/15 KSG: 23839/22 SYGN: V GNC UPR 206/14 DATA...,2022-08-31,8,784.79
6748,Pozostałe,760-001 | POZOSTAŁE PRZYCH.OPERACYJNE,FVPOZ FVP/22/08/0005 z dnia 10.08.2022 LIPIŃSKA,2022-08-10,8,"1,056.91"


## Wynik na działalności finansowej

In [84]:
# Działalność finansowa

t_tabela=t_konta_mis[(t_konta_mis['M']==msc) & ((t_konta_mis['P0'].str.startswith('E.'))| (t_konta_mis['P0'].str.startswith('F.')))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='WYNIK NA DFIN'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Kredyty inwestycyjne,-3.64,0.00,3.64,-100.00
Kredyty obrotowe,-7.11,-3.90,3.20,-45.09
Leasing,-11.28,-15.78,-4.50,39.85
Noty odsetkowe,-0.02,-0.79,-0.76,"3,062.50"
Odsetki,0.00,0.00,0.00,0.00
PFR,0.00,0.00,0.00,0.00
Pozostałe,0.00,0.00,0.00,0.00
Różnice kursowe,0.00,-0.02,-0.02,-inf
WFOŚiGW,0.00,0.00,0.00,0.00
WYNIK NA DFIN,-22.06,-20.49,1.57,-7.11


In [85]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '11_DF_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '11_DF_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

In [86]:
# Wykres

t_temp=t_konta_mis[((t_konta_mis['P0'].str.startswith('E.'))| (t_konta_mis['P0'].str.startswith('F.')))]
t_wykres=t_temp.groupby(['M'],as_index=False)[['P_2022','W_2022']].sum()
t_wykres=t_wykres[t_wykres['M']<=msc]

t_wykres['P_2022']=-t_wykres['P_2022']
t_wykres['W_2022']=-t_wykres['W_2022']

fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['P_2022'],  marker_symbol='circle-open',name="Plan",line= {"color": "gray", "width": 3}),row=1,col=1)
fig.add_trace(go.Scatter(x=t_wykres['M'], y=t_wykres['W_2022'],  marker_symbol='circle-open',name="Wykonanie",line= {"color": "green", "width": 3}),row=1,col=1)
fig.update_layout(title_text='Analiza odchyleń - Działalność finansowa (koszty)')
fig.update_layout(plot_bgcolor='#FFFFFF',font_size=12,title_font_size=14,width=700, height=450)
fig.write_image(IMAGES / '11_Finansowa.png')
fig.show()

In [87]:
# ANALIZA

t_analiza=t_zapisy[((t_zapisy['P0'].str.startswith('E.'))| (t_zapisy['P0'].str.startswith('F.')))]
t_analiza=t_analiza[['P2','Konto_N', 'Opis', 'Data','M', 'Kwota_PLN']]

t_analiza[t_analiza['M']==8].sort_values(by=['Kwota_PLN'],ascending=True)

,P2,Konto_N,Opis,Data,M,Kwota_PLN
11422,Leasing,751-003-05 | K.FIN.ODSETKI LEASINGOWE ŁADOWARKA CAT 980H (2020),CZYNSZ NR 29- CAT 980H 2461102208,2022-08-30,8,"-4,976.05"
7238,Kredyty obrotowe,751-001-06 | K.FIN.ODSETKI LINIA KREDYTOWA PEKAO,SPŁATA KREDYTU,2022-08-01,8,"-4,850.02"
9393,Kredyty obrotowe,751-001-06 | K.FIN.ODSETKI LINIA KREDYTOWA PEKAO,SPŁATA KREDYTU,2022-08-31,8,"-4,671.04"
10816,Leasing,751-003-04 | K.FIN. ODSETKI KRUSZARKA CH430 (2019),"CZYNSZ NR 35, KRUSZARKA CH430 2019 3471712207",2022-08-20,8,"-3,605.78"
10808,Leasing,751-003-09 | K.FIN.ODSETKI LEASINGOWE - MANIPULATOR Z MŁOTEM,RATA LEASINGOWA 17- MŁOT R2272/LO/86248/17/22,2022-08-20,8,"-1,839.13"
10397,Leasing,751-003-14 | K.FIN.ODSETKI LEASINGOWE - INSTALACJA FOTOWOLTAICZNA,RATA LEASINGOWA 2- INSTALACJA FOTOWOLTAICZNA FLO 22/08/02275,2022-08-01,8,"-1,721.93"
10710,Pozostałe,751-002-01 | KOSZTY FINANSOWE-POZOSTAŁE,PORĘCZENIE ZG Z UMOWĄ Z 21-12-2018r- DOT.POŻYCZKI Z WFOŚIGW DPU/22...,2022-08-31,8,"-1,640.00"
8187,Noty odsetkowe,751-001-01 | K.FIN.ODSETKI DLA DOSTAWCÓW KUP,NOTA ODSETKOWA NR CFRN.322.840.2022.DB- PKP CARGO,2022-08-31,8,"-1,355.14"
10626,Leasing,751-003-12 | K.FIN.ODSETKI LEASINGOWE - CIĄGNIK,RATA LEASINGOWA 16- CIĄGNIK 043941/07/2022/RL/LO/PLN,2022-08-07,8,-676.27
10836,Leasing,751-003-07 | K.FIN.ODSETKI LEASINGOWE- SKODA OCTAVIA IV(2020)- DSW...,ODSETKI RATA LEASINGU NR 17 SKODA OCTAVIA DW3MH48 107959/0822/RM,2022-08-20,8,-500.47


# NARASTAJĄCO

## Wynik finansowy

In [88]:
# Wynik finansowy

t_tabela=t_konta_mis[t_konta_mis['M']<=msc]
t_tabela=t_tabela.groupby(['P0'],as_index=False)[['P_2022','K_2022','Korekta','W_2022']].sum()
t_tabela.rename(columns={'P0':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]','K_2022':'K[2022]'},inplace=True)
t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='WYNIK (PBT) RAZEM      '
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]
t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela = t_tabela.replace({np.nan: ""})
t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}','K[2022]':'{0:,.2f}','Korekta':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],K[2022],Korekta,W[2022],Delta,Delta[proc]
A. Przychody netto ze sprzedaży,"40,325.48","48,069.27",0.00,"48,069.27","7,743.79",19.20
B. Koszty działalności operacyjnej,"-35,799.23","-43,518.68",0.00,"-43,518.68","-7,719.46",21.56
C. Pozostałe przychody operacyjne,0.00,"5,852.38",0.00,"5,852.38","5,852.38",inf
D. Pozostałe koszty operacyjne,0.00,"-6,062.60",0.00,"-6,062.60","-6,062.60",-inf
E. Przychody finansowe,0.00,0.00,0.00,0.00,0.00,inf
F. Koszty finansowe,-234.69,-197.60,0.00,-197.60,37.09,-15.80
WYNIK (PBT) RAZEM,"4,291.57","4,142.77",0.00,"4,142.77",-148.80,-3.47


In [89]:
# Tabela do raportu - księgowość

tabela=t_tabela[['Pozycja','K[2022]','Korekta']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '12_PBT_Księgowy.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '12_PBT_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '12_PBT_Odchylenia.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

## Przychody ze sprzedaży

In [90]:
# Przychody ze sprzedaży

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('A. Przychody'))]
t_tabela=t_tabela.groupby(['P1'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P1':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)
t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='PRZYCHODY RAZEM'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
1. Przychody netto,"40,325.48","47,601.83","7,276.35",18.04
2. Zmiana stanu produktów,0.00,-444.33,-444.33,-inf
3. Koszt wytworzenia prod. na wł.,0.00,-0.00,-0.00,-inf
4. Przychody netto TiM,0.00,911.77,911.77,inf
PRZYCHODY RAZEM,"40,325.48","48,069.27","7,743.79",19.20


In [91]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '13_Przychody_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '13_Przychody_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

## Koszty operacyjne

In [92]:
# Koszty operacyjne

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))]
t_tabela=t_tabela.groupby(['P1'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P1':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY OPERACYJNE RAZEM'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})
t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')


t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
1. Amortyzacja,"1,430.58","1,433.82",3.24,0.23
2. Zużycie materiałów i energii,"4,451.27","6,265.82","1,814.55",40.76
3.1. Usługi obce - transport,"7,064.61","11,058.45","3,993.84",56.53
3.2. Usługi obce - ekspl. złoża,"8,616.34","9,661.37","1,045.03",12.13
3.3. Usługi obce - bocznica,"3,385.09","3,287.23",-97.86,-2.89
3.4. Usługi obce - pozostałe,"3,795.84","3,745.19",-50.65,-1.33
4. Podatki i opłaty,"2,347.52","2,411.91",64.38,2.74
5. Wynagrodzenia,"3,187.08","3,335.93",148.85,4.67
6. Ubezpieczenia społeczne,833.00,786.07,-46.93,-5.63
7. Pozostałe koszty rodzajowe,687.90,631.15,-56.75,-8.25


In [93]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '14_KO_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '14_KO_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Zużycie Materiałów i Energii

In [94]:
# Koszty operacyjne - zużycie materiałów i energii

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Energia elektryczna,"1,830.00","2,953.70","1,123.70",61.40
Materiały biurowe,16.85,17.34,0.49,2.91
Materiały pozostałe,8.88,8.76,-0.12,-1.35
Materiały produkcyjne,"1,652.20","1,738.33",86.13,5.21
Paliwo,943.34,"1,547.68",604.34,64.06
KOSZTY - ZUŻYCIE MiE,"4,451.27","6,265.82","1,814.55",40.76


In [95]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_ZMiE_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_ZMiE_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Usługi obce transport

In [96]:
# Koszty operacyjne - usługi obce transport

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.1. Usługi obce - transport'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Transport kolejowy,"6,755.40","7,740.83",985.43,14.59
Transport samochodowy,309.21,"3,317.61","3,008.40",972.93
KOSZTY - ZUŻYCIE MiE,"7,064.61","11,058.45","3,993.84",56.53


In [97]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Transport_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Transport_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Obsługa złoza

In [98]:
# Koszty operacyjne - 

#3.1. Usługi obce - transport	
#3.2. Usługi obce - ekspl. złoża	
#3.3. Usługi obce - bocznica  	
#3.4. Usługi obce - pozostałe

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.2. Usługi obce - ekspl. złoża'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Badania i usługi pomiarowe,72.33,72.57,0.24,0.33
Dzierżawa gruntów,"1,060.36",964.80,-95.56,-9.01
Naprawy i remonty,392.63,432.89,40.26,10.25
Transport i zał. wewnętrzny,"4,503.04","5,022.11",519.07,11.53
Usługi wiertniczo - strzałowe,"1,918.51","2,341.04",422.53,22.02
Wynajem maszyn i urządzeń,669.46,827.95,158.49,23.67
KOSZTY - ZUŻYCIE MiE,"8,616.34","9,661.37","1,045.03",12.13


In [99]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Obsługa_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Obsługa_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Usługi obce - bocznica

In [100]:
# Koszty operacyjne - 

# Koszty operacyjne - 

#3.1. Usługi obce - transport	
#3.2. Usługi obce - ekspl. złoża	
#3.3. Usługi obce - bocznica  	
#3.4. Usługi obce - pozostałe

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.3. Usługi obce - bocznica'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Transport i obsługa bocznicy,"3,191.24","3,076.43",-114.81,-3.60
Wynajem bocznicy,193.85,210.80,16.95,8.74
KOSZTY - ZUŻYCIE MiE,"3,385.09","3,287.23",-97.86,-2.89


In [101]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Bocznica_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Bocznica_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Usługi obce poozstałe

In [102]:
# Koszty operacyjne - pozostałe

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('3.4. Usługi obce - pozostałe'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Leasing operacyjny,114.95,45.43,-69.52,-60.48
Monitoring TMS,17.68,18.09,0.41,2.32
Ochrona,236.39,219.65,-16.74,-7.08
"Poczta, telefon, internet",20.71,20.13,-0.58,-2.80
Pozostałe usługi,"3,335.86","3,359.49",23.63,0.71
"Telefon, internet",12.71,11.81,-0.90,-7.11
Usługi IT,57.54,70.59,13.05,22.68
KOSZTY - ZUŻYCIE MiE,"3,795.84","3,745.19",-50.65,-1.33


In [103]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Pozostałe_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_UO_Pozostałe_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Zużycie Materiałów i Energii

In [104]:
# Koszty operacyjne - zużycie materiałów i energii

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('2. Zużycie materiałów i energii'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUŻYCIE MiE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Energia elektryczna,"1,830.00","2,953.70","1,123.70",61.40
Materiały biurowe,16.85,17.34,0.49,2.91
Materiały pozostałe,8.88,8.76,-0.12,-1.35
Materiały produkcyjne,"1,652.20","1,738.33",86.13,5.21
Paliwo,943.34,"1,547.68",604.34,64.06
KOSZTY - ZUŻYCIE MiE,"4,451.27","6,265.82","1,814.55",40.76


In [105]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_ZMiE_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '15_KO_ZMiE_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Podatki i opłaty

In [106]:
# Koszty operacyjne - Podatki i opłaty

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('4. Podatki i opłaty'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - PODATKI I OPŁATY'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Opłata eksploatacyjna,"1,350.70","1,486.79",136.09,10.08
Opłaty środowiskowe,5.73,0.54,-5.19,-90.63
PFRON,37.05,50.40,13.35,36.02
Podatek leśny,0.46,0.46,0.00,0.00
Podatek od nieruchomości,351.57,356.37,4.80,1.37
Podatek rolny,0.56,0.56,0.00,0.00
Pozostałe opłaty,4.57,3.82,-0.75,-16.34
Uzytkowanie gruntów leśnych,568.54,494.39,-74.14,-13.04
Użytkowanie wieczyste,28.35,18.57,-9.77,-34.48
KOSZTY - PODATKI I OPŁATY,"2,347.52","2,411.91",64.38,2.74


In [107]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '16_KO_Podatki_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '16_KO_Podatki_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Wynagrodzenia

In [108]:
# Koszty operacyjne - Wynagrodzenia

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('5. Wynagrodzenia'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - WYNAGRODZENIA'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Ekwiwalenty i odprawy,-0.00,42.99,42.99,-inf
Nagrody jubileuszowe,-0.00,1.37,1.37,-inf
Rada nadzorcza,90.00,88.00,-2.00,-2.22
Rezerwa nagrody jubileuszowe,-0.00,-0.00,0.00,0.00
Rezerwa nagrody poozstałe,-0.00,-0.00,0.00,0.00
Rezerwa urlopowa,-0.00,0.00,0.00,-inf
Umowy o pracę,"2,790.23","2,858.00",67.77,2.43
Umowy zlecenia,135.13,158.55,23.42,17.33
Zarząd i Prokura,171.71,187.01,15.30,8.91
KOSZTY - WYNAGRODZENIA,"3,187.08","3,335.93",148.85,4.67


In [109]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '17_Wynagrodzenia_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '17_Wynagrodzenia_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Narzuty (ZUS)

In [110]:
# Koszty operacyjne - ZUS

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('6. Ubezpieczenia społeczne'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - ZUS I ŚWIADCZENIA'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Badania i opieka medyczna,16.60,9.30,-7.31,-44.00
Badania lekarskie,7.54,12.77,5.22,69.27
Ekwiwalenty,10.92,9.28,-1.64,-14.98
Fundusz socjalny,69.93,73.64,3.71,5.30
PPK,2.18,3.32,1.14,52.38
Rezerwa ZUS,-0.00,-0.00,-0.00,inf
Składki ZUS,616.07,594.23,-21.83,-3.54
Szkolenia,35.71,21.55,-14.16,-39.66
Świadczenia na rzecz pracowników,74.05,61.99,-12.07,-16.29
KOSZTY - ZUS I ŚWIADCZENIA,833.00,786.07,-46.93,-5.63


In [111]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '18_ZUS_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '18_ZUS__Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

### Koszty pozostałe

In [112]:
# Koszty operacyjne - pozostałe

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & (t_konta_mis['P0'].str.startswith('B. Koszty działalności operacyjnej '))& (t_konta_mis['P1'].str.startswith('7. Pozostałe koszty rodzajowe'))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)

t_tabela['P[2022]']=-t_tabela['P[2022]']
t_tabela['W[2022]']=-t_tabela['W[2022]']

t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='KOSZTY - POZOSTAŁE'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Fundusz Likwidacji Z. Górniczego,135.07,148.67,13.60,10.07
Podróże służbowe,11.66,18.05,6.39,54.84
Pozostałe koszty,1.63,2.20,0.56,34.47
Repreezntacja i reklama,60.46,35.83,-24.64,-40.75
Rozliczenie projektów rozwojowych,231.58,174.36,-57.23,-24.71
Ryczałty samochodowe,2.49,2.44,-0.05,-2.13
Samochody - nieodliczony VAT,48.13,40.07,-8.06,-16.74
Ubezpieczenia komunikacyjne,-0.00,24.75,24.75,-inf
Ubezpieczenia majątkowe,184.91,140.67,-44.24,-23.92
Ubezpieczenia wyrrobów gotowych,-0.00,34.43,34.43,-inf


In [113]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '19_KO_Pozostałe_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '19_KO_Pozostałe_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

## Działalność pozostała

In [114]:
# Pozostała działalność

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & ((t_konta_mis['P0'].str.startswith('C. Pozostałe przychody operacyjne'))| (t_konta_mis['P0'].str.startswith('D. Pozostałe koszty operacyjne')))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)
t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='WYNIK NA PDO'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Odpisy aktualizujące - zapasy,0.00,-52.15,-52.15,-inf
Odszkodowania i szkody,0.00,-0.91,-0.91,-inf
Pozostałe,0.00,-159.69,-159.69,-inf
Rekultywacja,0.00,-70.02,-70.02,-inf
Rozwiązanie rezerw,0.00,1.26,1.26,inf
Sprzedaż RAT,0.00,71.30,71.30,inf
WYNIK NA PDO,0.00,-210.22,-210.22,-inf


In [115]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '20_OOP_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '20_OOP_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

## Działalność finansowa

In [116]:
# Działalność finansowa

t_tabela=t_konta_mis[(t_konta_mis['M']<=msc) & ((t_konta_mis['P0'].str.startswith('E.'))| (t_konta_mis['P0'].str.startswith('F.')))]
t_tabela=t_tabela.groupby(['P2'],as_index=False)[['P_2022','W_2022']].sum()
t_tabela.rename(columns={'P2':'Pozycja','W_2022':'W[2022]','P_2022':'P[2022]'},inplace=True)
t_tabela=t_tabela.sort_values(by=['Pozycja'],ascending=True)
t_tabela.loc['RAZEM']=t_tabela.sum(numeric_only=True,axis=0)
t_tabela.loc['RAZEM','Pozycja']='WYNIK NA DFIN'
last_row = pd.IndexSlice[t_tabela.index[t_tabela.index == "RAZEM"], :]

t_tabela['Delta']=t_tabela['W[2022]']-t_tabela['P[2022]']
t_tabela['Delta[proc]']=(t_tabela['Delta']/t_tabela['P[2022]'])*100
t_tabela.fillna(0,inplace=True)
t_tabela = t_tabela.replace({np.nan: ""})

t_tabela_fv=t_tabela.style.format({'W[2022]':'{0:,.2f}','P[2022]':'{0:,.2f}','Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f}'}).hide(axis='index')

t_tabela_fv

Pozycja,P[2022],W[2022],Delta,Delta[proc]
Kredyty inwestycyjne,-29.12,0.00,29.12,-100.00
Kredyty obrotowe,-85.01,-45.08,39.92,-46.97
Leasing,-116.23,-133.70,-17.47,15.03
Noty odsetkowe,-4.33,-14.31,-9.98,230.33
Odsetki,0.00,0.00,0.00,inf
PFR,0.00,-0.02,-0.02,-inf
Pozostałe,0.00,-4.14,-4.14,-inf
Różnice kursowe,0.00,-0.04,-0.04,-inf
WFOŚiGW,0.00,-0.31,-0.31,-inf
WYNIK NA DFIN,-234.69,-197.60,37.09,-15.80


In [117]:
# Tabela do raportu - wartości

tabela=t_tabela[['Pozycja','P[2022]','W[2022]']].copy()
tabela=tabela.style.format({'P[2022]':'{0:,.2f}','W[2022]':'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / '21_DF_Wartości.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')

# Tabela do raportu - odchylenia

tabela=t_tabela[['Pozycja','Delta','Delta[proc]']].copy()
tabela=tabela.style.format({'Delta':'{0:,.2f}','Delta[proc]':'{0:,.2f} \%'}).hide(axis='index')
tabela.to_latex(ASSETS / '21_DF_Odchylenie.tex',hrules=True,column_format='L{5cm}R{2cm}R{2cm}')